## Evaluating the QASystem

### Test dataset of 5 sample questions and their expected answers and relevant source documents

In [4]:
# Example test cases
test_cases = [
    {
        "question": "What medications does Voy prescribe for weight loss?",
        "ideal_answer": "Voy prescribes GLP-1 medications like Wegovy and Mounjaro for eligible patients. These medications help regulate appetite and blood sugar levels. Voy's medical providers determine the most appropriate medication based on your medical history, current health status, and weight loss goals.",
        "relevant_ids":["https://joinvoy.zendesk.com/hc/en-gb/articles/32299448990356-Information-about-Wegovy","https://joinvoy.zendesk.com/hc/en-gb/articles/32242827773716-Information-about-Mounjaro"]
    },
    {
        "question": "How much does Voy's telehealth service cost?",
        "ideal_answer": "I don't have access to the costs of Voy's telehealth service.",
        "relevant_ids":[]
    },
    {
        "question": "Can I change dosage for medication after some time?",
        "ideal_answer": "Yes, you can remain on the same dose or reduce your dose throughout your treatment. You can adjust your dosage directly from your account. Based on your current dose, you'll have the option to increase, maintain, or lower the strength.",
        "relevant_ids":["https://joinvoy.zendesk.com/hc/en-gb/articles/34721699585300-How-do-I-change-my-dose"]
    },
    {
        "question": "How long does it take for the medication to arrive?",
        "ideal_answer": "Medication is dispatched after the consultation and prescription. Delivery times depend on your location.",
        "relevant_ids":["https://joinvoy.zendesk.com/hc/en-gb/articles/20199454690068-Do-you-ship-abroad", "https://joinvoy.zendesk.com/hc/en-gb/articles/20199416561172-Where-is-my-order"]
    },
    {
        "question": "Can I get a refund if I'm not satisfied?",
        "ideal_answer": "Yes orders have a cancellation window, before prescription is approved by the medical team. After this period, we cannot cancel order or accept returns",
        "relevant_ids":["https://joinvoy.zendesk.com/hc/en-gb/articles/20199844131348-What-is-your-Returns-policy"]
    },
]

In [5]:
# Load QA system
from qa_system import QASystem
qa_system = QASystem()

/Users/divyagautam/.pyenv/versions/ml_assignment_manual/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


System ready!


### 1. Checking how well is the retriever performing

Precision@k: The proportion of retrieved documents (up to rank k) that are relevant. For example, if you retrieve 5 documents (k=5) and 3 are relevant, Precision@5 is 3/5 = 0.6.

Precision@k = (# of relevant documents retrieved at or before rank k) / k

Recall@k: The proportion of all relevant documents that are retrieved (up to rank k). If there are 10 relevant documents in total, and you retrieve 3 of them within the top 5, Recall@5 is 3/10 = 0.3.

Recall@k = (# of relevant documents retrieved at or before rank k) / (Total # of relevant documents)

Mean Reciprocal Rank (MRR): Considers the rank of the first relevant document. The reciprocal rank is 1/rank. MRR averages these reciprocal ranks across multiple queries. Higher MRR is better.

MRR = (1/Q) * Σ (1 / rank_i) where Q is the number of queries and rank_i is the rank of the first relevant document for query i.

In [6]:
# Load the vectorestore from file
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2" 
embeddings = HuggingFaceEmbeddings(model_name=model_name)
vectorstore = InMemoryVectorStore.load(path="./inmemory_langchain_db", embedding=embeddings)

In [7]:
# function to find doc id for the urls
def lookup_id(url):
    for doc_id, doc in vectorstore.store.items():
        if 'metadata' in doc and 'url' in doc['metadata'] and doc['metadata']['url'].replace("\n", "") == url:
            return doc_id
    return None

In [8]:
from typing import Dict, List, Any
import itertools

def precisionRecallk(retrieved_doc_ids, relevant_ids, k):
    
    # Calculate precision and recall at k
    retrieved_at_k = retrieved_doc_ids[:k]
    relevant_set = set(relevant_ids)
    true_positives = sum(1 for doc_id in retrieved_at_k if doc_id in relevant_set)

    precision = true_positives / k if k > 0 else 0.0
    recall = true_positives / len(relevant_set) if len(relevant_set) > 0 else 0.0
    return precision, recall

def mrrCalc(retrieved_ids, relevant_ids):
    
    # Calculate the Mean Reciprocal Rank
    for i, doc_id in enumerate(retrieved_ids):
        if doc_id in relevant_ids:
            return 1.0 / (i + 1)
    return 0.0

In [9]:
all_precision_at_k = []
all_recall_at_k = []
all_mrr = []

k = [2,3,4]

for kk in k:
    print(f"Evaluating for k:", kk)
    for i, test_case in enumerate(test_cases):
        
        question = test_case["question"]
        relevant_document_ids = list(lookup_id(url) for url in test_case["relevant_ids"]) # List of doc id's

        retriever = vectorstore.as_retriever(
            search_type="mmr", search_kwargs={"k": kk})
        
        result = retriever.invoke(question) # Uses the run_query from the main code
        retrieved_doc_ids = list(lookup_id(res.metadata['url']) for res in result)

        precision_at_k, recall_at_k = precisionRecallk(retrieved_doc_ids, relevant_document_ids, kk)
        mrr =  mrrCalc(retrieved_doc_ids, relevant_document_ids)

        all_precision_at_k.append(precision_at_k)
        all_recall_at_k.append(recall_at_k)
        all_mrr.append(mrr)

        print("Question {}: {}".format(i+1, test_case["question"]))
        print(f"Precision @ {kk}: {precision_at_k:.3f}")
        print(f"Recall @ {kk}: {recall_at_k:.3f}")
        print(f"MRR: {mrr:.3f}")
        print("---")

    avg_precision_at_k = sum(all_precision_at_k) / len(all_precision_at_k)
    avg_recall_at_k = sum(all_recall_at_k) / len(all_recall_at_k)
    avg_mrr = sum(all_mrr) / len(all_mrr)

    print(f"Average Precision @ {kk}: {avg_precision_at_k:.3f}")
    print(f"Average Recall @ {kk}: {avg_recall_at_k:.3f}")
    print(f"Average MRR: {avg_mrr:.3f}")
    print("\n")

Evaluating for k: 2
Question 1: What medications does Voy prescribe for weight loss?
Precision @ 2: 0.000
Recall @ 2: 0.000
MRR: 0.000
---
Question 2: How much does Voy's telehealth service cost?
Precision @ 2: 0.000
Recall @ 2: 0.000
MRR: 0.000
---
Question 3: Can I change dosage for medication after some time?
Precision @ 2: 1.000
Recall @ 2: 2.000
MRR: 1.000
---
Question 4: How long does it take for the medication to arrive?
Precision @ 2: 1.000
Recall @ 2: 1.000
MRR: 1.000
---
Question 5: Can I get a refund if I'm not satisfied?
Precision @ 2: 0.000
Recall @ 2: 0.000
MRR: 0.000
---
Average Precision @ 2: 0.400
Average Recall @ 2: 0.600
Average MRR: 0.400


Evaluating for k: 3
Question 1: What medications does Voy prescribe for weight loss?
Precision @ 3: 0.000
Recall @ 3: 0.000
MRR: 0.000
---
Question 2: How much does Voy's telehealth service cost?
Precision @ 3: 0.000
Recall @ 3: 0.000
MRR: 0.000
---
Question 3: Can I change dosage for medication after some time?
Precision @ 3: 1.

### 2. Checking the semantic similarity scoring between LLM responses and source documents using metrics like cosine similarity to measure alignment with reference answers

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

def qaSystemResponseSimilarity(response, reference_answer):
    
    # Calculate semantic similarity
    model = SentenceTransformer('all-MiniLM-L6-v2')
    response_embedding = model.encode(response)
    reference_embedding = model.encode(reference_answer)
    similarity = util.pytorch_cos_sim(response_embedding, reference_embedding).item()
    
    return similarity

for i, test_case in enumerate(test_cases):
    response = qa_system.answer_question(test_case["question"])
    print("Question {}: {}".format(i+1, test_case["question"]))
    print("Ideal Answer:", test_case["ideal_answer"])
    print("LLM Response:", response)
    
    evaluation = qaSystemResponseSimilarity(response, test_case["ideal_answer"])
    print("Semantic Similarity:", evaluation)
    print("\n")

Question 1: What medications does Voy prescribe for weight loss?
Ideal Answer: Voy prescribes GLP-1 medications like Wegovy and Mounjaro for eligible patients. These medications help regulate appetite and blood sugar levels. Voy's medical providers determine the most appropriate medication based on your medical history, current health status, and weight loss goals.
LLM Response: I don't have enough information to answer that question.
Semantic Similarity: 0.03991740942001343


Question 2: How much does Voy's telehealth service cost?
Ideal Answer: I don't have access to the costs of Voy's telehealth service.
LLM Response: I don't have enough information to answer that question.
Semantic Similarity: 0.22662264108657837


Question 3: Can I change dosage for medication after some time?
Ideal Answer: Yes, you can remain on the same dose or reduce your dose throughout your treatment. You can adjust your dosage directly from your account. Based on your current dose, you'll have the option to 

### 3. Evaluating the QASystem against certain criteria

Criteria can be anything that a human would consider important to check. LabeledCriteriaEvalChain evaluates responses against 3 criteria, described as medical accuracy, safe advice and completeness. These can be tailed to domain-specific requirements and can be made more stringent.

In [ ]:
from langchain.evaluation.criteria import LabeledCriteriaEvalChain
from langchain_google_genai import GoogleGenerativeAI
from utils.rag import convert_lc_docs
import json

def qaSystemFactChecking(question, source_docs):
    result = qa_system.answer_question(question)
    answer = result
    
    # Medical fact checking criteria
    criteria = {
        "medical_accuracy": "Does the response contain any medical claims not supported by the reference documents?",
        "safe_advice": "Does the response provide any medical advice that could be harmful if followed?",
        "completeness": "Does the response cover all relevant safety information from the reference documents?"
    }
    
    criteria_evaluator = LabeledCriteriaEvalChain.from_llm(
        llm=GoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05", temperature=0),
        criteria=criteria
    )
    
    criteria_result = criteria_evaluator.evaluate_strings(
        prediction=answer,
        input=question,
        reference="\n".join([doc.page_content for doc in source_docs])
    )
    print(criteria_result)
    return answer, criteria_result
    
all_faqs = json.load(open("./outputs/all_faqs.json", "r"))
source_docs = convert_lc_docs(all_faqs)
    
for i, test_case in enumerate(test_cases[:3]):
    print("Question {}: {}".format(i+1, test_case["question"]))
    
    response, evaluation_results = qaSystemFactChecking(test_case["question"], source_docs)
    print("LLM Response:", response)
    print("Evaluation summary:", " ".join(evaluation_results['reasoning'].splitlines()))
    print("Criteria met?:", evaluation_results['value'])
    print("Score:", evaluation_results['score'])
    print("\n")

Question 1: What medications does Voy prescribe for weight loss?
{'reasoning': 'The submission states, "I don\'t have enough information to answer that question." Let\'s analyze this against the provided criteria:\n\n1.  **medical_accuracy:** The submission makes no medical claims whatsoever. It simply states a lack of information. Therefore, it cannot contain any unsupported medical claims.\n\n2.  **safe_advice:** The submission offers no medical advice, so it cannot provide harmful advice.\n\n3.  **completeness:** The question asks about medications Voy prescribes. The reference material discusses the medications Wegovy and Mounjaro, and how to start at a higher dose if already taking GLP-1 medication. It also explains why specific prescription medications are not mentioned upfront. The submission, by stating it doesn\'t have enough information, fails to address the relevant information in the reference documents.\n\nThe submission is medically accurate and safe, but it is not comple